# Exercise 3

## Initializing the spark context


In [1]:
val scalaVersion = scala.util.Properties.versionNumberString

scalaVersion: String = "2.11.8"

In [2]:
classpath.add("org.apache.spark" %% "spark-core" % "1.6.0")

130 new artifact(s)

130 new artifacts in macro


130 new artifacts in runtime
130 new artifacts in compile


In [3]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext

@transient val sc = {
  val conf = new SparkConf()
      .setAppName("Test1")
      .setMaster("local[*]")
  SparkContext.getOrCreate(conf)
}


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
16/05/18 12:40:58 INFO SparkContext: Running Spark version 1.6.0
16/05/18 12:40:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/18 12:40:58 INFO SecurityManager: Changing view acls to: mtrampont
16/05/18 12:40:58 INFO SecurityManager: Changing modify acls to: mtrampont
16/05/18 12:40:58 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users with view permissions: Set(mtrampont); users with modify permissions: Set(mtrampont)
16/05/18 12:40:58 INFO Utils: Successfully started service 'sparkDriver' on port 55131.
16/05/18 12:40:59 INFO Slf4jLogger: Slf4jLogger started
16/05/18 12:40:59 INFO Remoting: Starting remoting
16/05/18 12:40:59 INFO Remoting: Remoting started; listening on addresses :[akka.tcp://sparkDriverActorSystem@192.168.56.1:55144]
16/05/18 12:40:59 INFO Utils: Successfully start

import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@1521e7cd

## Parsing the csv file

### Into an RDD
Here is we parse the file using only spark to create an RDD with tuples containing the search date and destination.

In [4]:
val csvPath = "d:/workspace/TID_coding_tests/coding_tests/src/main/resources/searches.csv"

csvPath: String = "d:/workspace/TID_coding_tests/coding_tests/src/main/resources/searches.csv"

Extracting the header line and the columns indexes.

In [5]:
import scala.io.Source

val headerLine =
  Source.fromFile(csvPath)
        .getLines()
        .take(1)
        .toList
        .head

val colNames =
  headerLine.split("""\^""", -1)
      .map(_.trim)
      .zipWithIndex
      .toMap


import scala.io.Source
headerLine: String = "Date^Time^TxnCode^OfficeID^Country^Origin^Destination^RoundTrip^NbSegments^Seg1Departure^Seg1Arrival^Seg1Date^Seg1Carrier^Seg1BookingCode^Seg2Departure^Seg2Arrival^Seg2Date^Seg2Carrier^Seg2BookingCode^Seg3Departure^Seg3Arrival^Seg3Date^Seg3Carrier^Seg3BookingCode^Seg4Departure^Seg4Arrival^Seg4Date^Seg4Carrier^Seg4BookingCode^Seg5Departure^Seg5Arrival^Seg5Date^Seg5Carrier^Seg5BookingCode^Seg6Departure^Seg6Arrival^Seg6Date^Seg6Carrier^Seg6BookingCode^From^IsPublishedForNeg^IsFromInternet^IsFromVista^TerminalID^InternetOffice"
colNames: Map[String, Int] = Map(
  "TxnCode" -> 2,
  "Seg6Departure" -> 34,
  "TerminalID" -> 43,
  "Country" -> 4,
  "Seg1Carrier" -> 12,
  "Seg3Departure" -> 19,
  "Seg5Departure" -> 29,
  "Origin" -> 5,
  "RoundTrip" -> 7,
  "OfficeID" -> 3,
  "Seg2Arrival" -> 15,
  "InternetOffice" -> 44,
  "Seg5Carrier" -> 32,
  "Seg3Date" -> 21,
  "Seg5BookingCode" -> 33,
  "Seg2Departure" -> 14,
  "Seg1BookingCode" -> 13,
  "Seg4D

Now we can parse the file and create a RDD[(LocalDate, String)] with the search date and the destination airport.
I use a scala wrapper around Joda time library to do the parsing and get immutable LocaDate.

In [6]:
classpath.add( "com.github.nscala-time" %% "nscala-time" % "1.4.0")


3 new artifact(s)

3 new artifacts in macro


3 new artifacts in runtime
3 new artifacts in compile


In [7]:
import scala.util.{Try, Failure, Success}
import org.joda.time.format.ISODateTimeFormat

val searches =
  sc.textFile(csvPath)
    .filter(_ != headerLine) // The first partition will get the header line. We need to discard it
    .map{ record =>
      Try{
        val fields = record.split("""\^""", -1).map(_.trim)
        val dateFormat = ISODateTimeFormat.date()
        dateFormat.parseLocalDate(fields(colNames("Date"))) -> fields(colNames("Destination"))
      }.recoverWith{ case cause =>
        Failure(new IllegalArgumentException(s"Parsing failed with line: $record", cause))
      }
    }

searches.collect{case Failure(cause) => cause}
    .collect()
    .foreach(println)

java.lang.IllegalArgumentException: Parsing failed with line: 2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,,,,,,,,,,,,,,,,,,,,,,,,,,,,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
java.lang.IllegalArgumentException: Parsing failed with line: 2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,,,,,,,,,,,,,,,,,,,,,,,,,,,,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
java.lang.IllegalArgumentException: Parsing failed with line: 2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,,,,,,,,,,,,,,,,,,,,,,,,,,,,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
java.lang.IllegalArgumentException: Parsing failed with line: 2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,,,,,,,,,,,,,,,,,,,,,,,,,,,,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
java.lang.IllegalArgumentException: Parsing failed with line: 2013-01-01,10:15:3

import scala.util.{Try, Failure, Success}
import org.joda.time.format.ISODateTimeFormat
searches: org.apache.spark.rdd.RDD[util.Try[(org.joda.time.LocalDate, String)]] = MapPartitionsRDD[3] at map at Main.scala:34

Many lines are still comma-separated. We could recover them. Let's just drop them for now.

Let's start with a version using hardcoded airport codes for Madrid, Malaga, and Barcelona.

In [8]:
val keyAirports = Set("MAD", "AGP", "BCN")

keyAirports: Set[String] = Set("MAD", "AGP", "BCN")

Now we extract the month of each search and aggregate by (month, destination).

In [9]:
val monthlySearches =
  searches.collect{ case Success((date, dest)) => (date.withDayOfMonth(1), dest) -> date }
          .filter{ case ((_, dest), _) => keyAirports.contains(dest) }
          .aggregateByKey(0)((total, searchDate) => total + 1, _+_)

monthlySearches: org.apache.spark.rdd.RDD[((org.joda.time.LocalDate, String), Int)] = ShuffledRDD[7] at aggregateByKey at Main.scala:30

Let's group them by destination and collect them.

In [10]:
val monthlySearchesByDest =
  monthlySearches.map{ case ((month, dest), nbSearches) => dest -> (month, nbSearches)}
                 .groupByKey()
                 .collectAsMap()

monthlySearchesByDest: collection.Map[String, Iterable[(org.joda.time.LocalDate, Int)]] = Map(
  "AGP" -> CompactBuffer(
    (2013-04-01, 8265),
    (2013-07-01, 8892),
    (2013-11-01, 6384),
    (2013-06-01, 7923),
    (2013-03-01, 10659),
    (2013-01-01, 9633),
    (2013-10-01, 6499),
    (2013-09-01, 8151),
    (2013-12-01, 3696),
    (2013-05-01, 10830),
    (2013-02-01, 8379),
    (2013-08-01, 7866)
  ),
  "MAD" -> CompactBuffer(
    (2013-07-01, 22971),
    (2013-08-01, 21831),
    (2013-12-01, 14504),
    (2013-09-01, 21147),
...

We need to sort these by month before plotting.

In [11]:
import com.github.nscala_time.time.Imports._
import scala.collection.immutable.SortedMap

val searchesByDest = monthlySearchesByDest.mapValues(SortedMap.empty[LocalDate, Int] ++ _)

import com.github.nscala_time.time.Imports._
import scala.collection.immutable.SortedMap
searchesByDest: collection.Map[String, collection.immutable.SortedMap[com.github.nscala_time.time.Imports.LocalDate, Int]] = Map(
  "AGP" -> Map(
    2013-01-01 -> 9633,
    2013-02-01 -> 8379,
    2013-03-01 -> 10659,
    2013-04-01 -> 8265,
    2013-05-01 -> 10830,
    2013-06-01 -> 7923,
    2013-07-01 -> 8892,
    2013-08-01 -> 7866,
    2013-09-01 -> 8151,
    2013-10-01 -> 6499,
    2013-11-01 -> 6384,
    2013-12-01 -> 3696
  ),
  "MAD" -> Map(
    2013-01-01 -> 24258,
    2013-02-01 -> 22800,
    2013-03-01 -> 24681,
    2013-04-01 -> 25251,
...

Now let's try to use breeze-viz to plot that.

In [12]:
classpath.add("org.scalanlp" %% "breeze-viz" % "0.12")

18 new artifact(s)

18 new artifacts in macro


18 new artifacts in runtime
18 new artifacts in compile


In [13]:
import breeze.plot._


import breeze.plot._

In [14]:
val searchFig = Figure("Monthly searches")

searchFig: Figure = breeze.plot.Figure@2dd09f33

In [15]:
import org.joda.time.Months

val firstMonth = searchesByDest.flatMap{case (dest, srch) => srch.keys}.min

val p = searchFig.subplot(0)
searchesByDest.foreach{ case (dest, monthSearches) =>
  p += plot(
    x = monthSearches.keys.toIndexedSeq.map(Months.monthsBetween(firstMonth,_).getMonths),
    y = monthSearches.values.toIndexedSeq,
    name = dest
  )
}
searchFig.refresh()
                       

import org.joda.time.Months
firstMonth: LocalDate = 2013-01-01
p: Plot = breeze.plot.Plot@7760418

On the figure, the x-axis is displaying the number of months since the first month because it seems breeze-viz API does not manage TimeSeriesDataset so far.

Let's try to add a few labels.

In [16]:
p.title = "Monthly searches"
p.xlabel = "Months"
p.ylabel = "Nb searches"
p.legend = true

searchFig.refresh()

In [17]:
sc.stop()